# Model: Trackster neighborhood approach

Input:
- graph of tracksters in the cone neighbourhood.
- list of edges
- label 1 or 0 for each edge

Options:
- use EdgeConv operators to extract information from the neighborhood
    - then predict binary output per edge
- try: use DGCNN to let the network make its own edges in the latent space
    - force the original edges in the last conv layer to get the same output?

Use the batch trick to encode multiple samples at once (need to reindex edges).
- start with batch size 1 to make this easier

Using torch_geometric here as it's much easier.

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import random_split
from torch.optim.lr_scheduler import CosineAnnealingLR

import numpy as np
import matplotlib.pyplot as plt

from torch_geometric.nn import EdgeConv, DynamicEdgeConv
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
import torch_geometric.utils as geo_utils

import sklearn.metrics as metrics

from reco.learn import train_edge_pred, test_edge_pred
from reco.dataset import TracksterGraph
from reco.loss import FocalLoss

ds_name = "CloseByTwoPion"

# data_root = "data"
# raw_dir = f"/Users/ecuba/data/{ds_name}"

data_root = "/mnt/ceph/users/ecuba/processed"
raw_dir = f"/mnt/ceph/users/ecuba/{ds_name}"

In [81]:
transform = T.Compose([T.NormalizeFeatures()])

ds = TracksterGraph(
    ds_name,
    data_root,
    raw_dir,
    N_FILES=500,
    MAX_DISTANCE=10,
    ENERGY_THRESHOLD=10,
    include_graph_features=False,
    transform=transform,
)
ds

TrackstersGraph(graphs=49929, nodes=1406097, edges=5088768, max_distance=10, energy_threshold=10, graph_features=False)

In [ ]:
# use a custom scaler so we can reuse it in evaluation
scaler = StandardScaler()
scaler.fit(ds.data.x)
ds.x = torch.tensor(scaler.transform(ds.x)).type(torch.float)

In [82]:
ds_size = len(ds)
test_set_size = ds_size // 10
train_set_size = ds_size - test_set_size
train_set, test_set = random_split(ds, [train_set_size, test_set_size])
print(f"Train graphs: {len(train_set)}, Test graphs: {len(test_set)}")

# this is very nice - handles the dimensions automatically
train_dl = DataLoader(train_set, batch_size=64, shuffle=True)
test_dl = DataLoader(test_set, batch_size=64, shuffle=True)

Train graphs: 44937, Test graphs: 4992


In [83]:
print(f"dataset balance: {float(sum(ds.data.y) / len(ds.data.y)):.3f}") 

dataset balance: 0.302


In [84]:
# CUDA Setup
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [88]:
# ParticleNet

class EdgeConvBlock(nn.Module):

    def __init__(self, input_dim, hidden_dim, aggr="add", skip_link=False, k=16):
        super(EdgeConvBlock, self).__init__()

        convnetwork = nn.Sequential(
            nn.Linear(2 * input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )

        self.graphconv = EdgeConv(nn=convnetwork, aggr=aggr)
        self.dynamicgraphconv = DynamicEdgeConv(nn=convnetwork, aggr=aggr, k=k)
        self.skip_link = skip_link
        
    def forward(self, X, edge_index=None):
        
        if edge_index is None:
            H = self.dynamicgraphconv(X)
        else:
            H = self.graphconv(X, edge_index)

        if self.skip_link:
            return torch.hstack((H, X))

        return H



class GraphNet(nn.Module):
    def __init__(self, input_dim, output_dim=1, aggr='add', dropout=0.2, skip_link=False):
        """
        skip_link might not make so much difference if the edges are fixed
        """
        
        super(GraphNet, self).__init__()

        hdim1 = 64
        in_dim2 = hdim1 + input_dim if skip_link else hdim1
        
        hdim2 = 128
        in_dim3 = hdim2 + in_dim2 if skip_link else hdim2

        hdim3 = 256
        in_dim4 = hdim3 + in_dim3 if skip_link else hdim3

        # EdgeConv
        self.graphconv1 = EdgeConvBlock(input_dim, hdim1, skip_link=skip_link)
        self.graphconv2 = EdgeConvBlock(in_dim2, hdim2, skip_link=skip_link)
        self.graphconv3 = EdgeConvBlock(in_dim3, hdim3, skip_link=skip_link)

        # Edge features from node embeddings for classification
        self.edgenetwork = nn.Sequential(
            nn.Linear(2 * in_dim4, hdim3),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hdim3, output_dim),
            nn.Sigmoid()
        )
            
    def forward(self, X, edge_index):        
        # (prepared_edges, _) = geo_utils.add_self_loops(edge_index)  
        undirected_index = geo_utils.to_undirected(edge_index)

        H = self.graphconv1(X, undirected_index)
        H = self.graphconv2(H)
        H = self.graphconv3(H)
        
        src, dst = edge_index
        return self.edgenetwork(torch.cat([H[src], H[dst]], dim=-1)).squeeze(-1)

In [ ]:
model = GraphNet(input_dim=ds.data.x.shape[1], skip_link=False)
epochs = 100

# loss_func = F.binary_cross_entropy_with_logits
# alpha - percentage of negative edges
loss_func = FocalLoss(alpha=0.3, gamma=2)

model = model.to(device)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, epochs, eta_min=1e-3)

for epoch in range(epochs):

    train_loss, train_true, train_pred = train_edge_pred(
        model,
        device,
        optimizer,
        loss_func,
        train_dl
    )
    
    train_acc = metrics.accuracy_score(train_true, (train_pred > 0.5).astype(int))
    scheduler.step()

    if epoch % 2 == 0:
        test_loss, test_true, test_pred = test_edge_pred(model, device, loss_func, test_dl)
        test_acc = metrics.accuracy_score(test_true, (test_pred > 0.5).astype(int))
        print(
            f"Epoch {epoch}:",
            f"\ttrain loss:{train_loss:.2f}\ttrain acc: {train_acc:.3f}",
            f"\t test loss:{test_loss:.2f} \t test acc: {test_acc:.3f}"
        )

Epoch 0: 	train loss:1705.76	train acc: 0.814 	 test loss:78.36 	 test acc: 0.856
Epoch 2: 	train loss:379.34	train acc: 0.890 	 test loss:31.39 	 test acc: 0.901
Epoch 4: 	train loss:250.96	train acc: 0.907 	 test loss:22.28 	 test acc: 0.914
Epoch 6: 	train loss:203.41	train acc: 0.914 	 test loss:18.84 	 test acc: 0.918
Epoch 8: 	train loss:177.27	train acc: 0.918 	 test loss:16.65 	 test acc: 0.923
Epoch 10: 	train loss:161.99	train acc: 0.921 	 test loss:14.89 	 test acc: 0.926
Epoch 12: 	train loss:151.27	train acc: 0.922 	 test loss:14.27 	 test acc: 0.927
Epoch 14: 	train loss:142.43	train acc: 0.924 	 test loss:13.77 	 test acc: 0.927
Epoch 16: 	train loss:135.20	train acc: 0.925 	 test loss:13.03 	 test acc: 0.928
Epoch 18: 	train loss:130.76	train acc: 0.926 	 test loss:13.02 	 test acc: 0.929
Epoch 20: 	train loss:126.51	train acc: 0.927 	 test loss:12.00 	 test acc: 0.931
Epoch 22: 	train loss:123.02	train acc: 0.928 	 test loss:12.11 	 test acc: 0.930
Epoch 24: 	train los

In [ ]:
th_values = [i / 100. for i in range(1, 99)]
precision = []
recall = []
fbeta = []
accuracy = []

for th in th_values:
    test_loss, test_true, test_pred = test_edge_pred(model, device, loss_func, test_dl)

    pred = (test_pred > th).astype(int)

    if sum(pred) == 0:
        precision.append(0)
        recall.append(0)
        fbeta.append(0)
        accuracy.append(0)
    else:
        accuracy.append(metrics.accuracy_score(test_true, pred))
        precision.append(metrics.precision_score(test_true, pred))
        recall.append(metrics.recall_score(test_true, pred))
        fbeta.append(metrics.fbeta_score(test_true, pred, beta=1))

plt.figure()
plt.plot(th_values, precision, label="precision")
plt.plot(th_values, recall, label="recall")
plt.plot(th_values, fbeta, label="fbeta")
plt.plot(th_values, accuracy, label="accuracy")
plt.xlabel("Threshold")
plt.legend()
plt.show()

decision_th = th_values[np.argmax(fbeta)]
print(f"Th: {decision_th} | F-score: {max(fbeta):.3f} | accuracy: {accuracy[np.argmax(fbeta)]:.3f}")

In [ ]:
torch.save(model.state_dict(), f"models/DynamicParticleNet_64_128_256_noskip-{epochs}e-CloseByTwoPion_10_10_ngf_{ds.N_FILES}f.pt")

## Scoreboard
- ParticleNet_64_128_256_skip
    - Best F-score: 0.902
    - Accuracy: 0.937
    
 - ParticleNet_64_128_256_noskip
     - Best F-score: 0.902
     - accuracy: 0.937
     
     
 - DynamicParticleNet_64_128_256_noskip (this thing is super slow)



# Evaluation

In [ ]:
import uproot
import numpy as np
from reco.evaluation import graph_model_evaluation

file_name = f"{raw_dir}/new_ntuples_15101852_455.root"
tracksters = uproot.open({file_name: "ticlNtuplizer/tracksters"})
simtracksters = uproot.open({file_name: "ticlNtuplizer/simtrackstersSC"})
graphs = uproot.open({file_name: "ticlNtuplizer/graph"})
associations = uproot.open({file_name: "ticlNtuplizer/associations"})

In [ ]:
result = graph_model_evaluation(
    tracksters,
    simtracksters,
    associations,
    graphs,
    model.to("cpu"),
    decision_th,
    max_distance=10,
    energy_threshold=10,
    max_events=10,
)